<a href="https://colab.research.google.com/github/sindhu819/Projects/blob/master/Friends_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import numpy as np
import pandas as pd

from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import IDF

from pyspark.ml import Pipeline, PipelineModel


from pyspark.sql.functions import *

from pyspark.sql.types import *

import folium
import html

In [0]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")


from pyspark.sql.functions import *
from pyspark.sql.types import *



import folium
import html

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
users_file=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Users.csv'
users=spark.read.csv(users_file,header=True)

In [0]:
business_file=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Business.csv'
business=spark.read.csv(business_file,header=True)


In [0]:
review_file=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Reviews.parquet'
reviews=spark.read.parquet(review_file)

In [0]:
friends_file=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Friends*'
friends=spark.read.parquet(friends_file)
print(friends_file)

/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Friends*


In [0]:
friends.count()

29166583

In [0]:
friends.take(6)

[Row(user_id='bGAMUHYGMIzc30ydvD3v5w', friend_id=' sIf39JI3PibwqpTr0ErFEg'),
 Row(user_id='fFGs67RhFZEsSPEXT_5wpQ', friend_id=' 4H0YYvHsabRrpTG4xUpV-A'),
 Row(user_id='BNuPibT74jf8Kp47bZb7DQ', friend_id=' -DmqnhW4Omr3YhmnigaqHg'),
 Row(user_id='1yAa95hjob9E7ii-J0d_CQ', friend_id=' aqs1fjYP4xzRy6QPynDSDQ'),
 Row(user_id='4ZfcCa4m5RWvO4EFzfYm1A', friend_id=' 6rzu9rpASB3GCmt9Gm7jkw'),
 Row(user_id='-ia1ApLFMIbMjF4r3aaraQ', friend_id='9W3FNHyvrrIplqIucAB2ag')]

In [0]:
business.createOrReplaceTempView("business")
users.createOrReplaceTempView("users")
reviews.createOrReplaceTempView("reviews")
friends.createOrReplaceTempView("friends")

In [0]:
spark.sql("select user_id,count(*) from reviews group by user_id order by count(*) desc limit 100").show(100,False)

+----------------------+--------+
|user_id               |count(1)|
+----------------------+--------+
|bLbSNkLggFnqwNNzzq-Ijw|1247    |
|PKEzKWv_FktMm2mGPjwd0Q|851     |
|U4INQZOPSUaj8hMjLlZ3KA|694     |
|UYcmGbelzRa0Q6JqzLoguw|636     |
|n86B7IkbU20AkxlFX_5aew|585     |
|3nDUQBjKyVor5wV0reJChg|536     |
|L8P5OWO1Jh4B2HLa1Fnbng|508     |
|C2C0GPKvzWWnP57Os9eQ0w|503     |
|qewG3X2O4X6JKskxyyqFwQ|460     |
|8DEyKVyplnOcSKx39vatbg|439     |
|N3oNEwh0qgPqPP3Em6wJXw|414     |
|JaqcCU3nxReTW2cBLHounA|393     |
|tH0uKD-vNwMoEc3Xk3Cbdg|382     |
|oeAhRa8yFa9jtrhaHnOyxQ|356     |
|s2o_JsABvrZVm_T03qrBUw|351     |
|_VMGbmIeK71rQGwOBWt_Kg|340     |
|48vRThjhuhiSQINQ2KV8Sw|318     |
|eZZyuJDouIg4p-GYB3PV_A|313     |
|rt3PC7WCgCKsoufmQJELfw|308     |
|FIk4lQQu1eTe2EpzQ4xhBA|307     |
|y3FcL4bLy0eLlkb0SDPnBQ|306     |
|iSC96O2NjQc3JExGUHQG0Q|301     |
|hZfzVrhsCQ9JDAb2jYoJNQ|299     |
|RQlnSCjuqMnhR3Qk6j4KoA|292     |
|o68A_3bAJTge-ioGXHdxKA|291     |
|3Vd_ATdvvuVVgn_YCpz8fw|290     |
|YMgZqBUAddmFE

In [0]:
friends.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- friend_id: string (nullable = true)



In [0]:
spark.sql("select user_id from friends where friend_id='bLbSNkLggFnqwNNzzq-Ijw'").show(10,False)

+----------------------+
|user_id               |
+----------------------+
|5sNyAM0gX8pZyS867k6vdw|
|Hhmwl8u4ZxkBW1mGnJEUxw|
|FbN4sKq9n6j00st25Mc1PA|
|KP29qdHjCwXKFNVjrT-G2Q|
|swC93y0l10W-Sz8HxRSU0A|
|WMhgjmJPSuTFL381tVEZRg|
|iBtIOf5SrJdV0UPg4sxoEg|
|c7fMALatVC5_lzgeP88FlQ|
|IPGCGImT5tsLWWpLbvmLZQ|
|qzz9Q-hKfOqgqoNncXr7MQ|
+----------------------+
only showing top 10 rows



In [0]:
u_id = '5sNyAM0gX8pZyS867k6vdw'

query = """
select business_id, count(*) as 4_5_stars_count 
from reviews
where user_id in
    (select f.friend_id from friends f
    inner join users u on f.friend_id = u.user_id
    where f.user_id = "{}") 
and stars >= 4
and business_id not in (select business_id from reviews where user_id = "{}")
group by business_id
order by count(*) desc limit 100
""".format(u_id, u_id)

friend_recoms_df = spark.sql(query)

friend_recoms_df.toPandas()

,business_id,4_5_stars_count
0,mz1zD5_eqyCQNuzAqB_iKg,2
1,zpoZ6WyQUYff18-z4ZU1mA,2
2,O_C-2kZMHGnq3ZDL_OaG6Q,2
3,D6K3VT6S8FwFm4u5uN8T8g,2
4,OHFH6ALQqq35NiEbD1eXuw,2
...,...,...
95,LklWSvyLZXN3tEGxT8rIfg,1
96,CSaVK9wgqe2_tQuuCDINyQ,1
97,vL9L9NZ-LOVXeA4XuwNz1g,1
98,dzKDFiWLkspVBJKGfT0M2g,1


In [0]:

friend_recoms_df = friend_recoms_df.sample(False, 0.5).limit(10)
friend_recoms_df.toPandas()

,business_id,4_5_stars_count
0,PYrA0thkuLCVAknUHYhVEQ,2
1,D6K3VT6S8FwFm4u5uN8T8g,2
2,mz1zD5_eqyCQNuzAqB_iKg,2
3,OHFH6ALQqq35NiEbD1eXuw,2
4,364hhL5st0LV16UcBHRJ3A,2
5,8SwMaPL6AJIs1ZPSZ3NJ4w,2
6,MgSd4P3ATkiywbbraFygHg,1
7,rbH0HuTcBvZBbiAnqufRqQ,1
8,FS0SA_ysgXsQEe6aCaIrKQ,1
9,0YG_kM5caZhtwd6nbnei6A,1


In [0]:
def getBusinessDetails(in_bus):
    
    a = in_bus.alias("a")
    b = business.alias("b")
    
    return a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.business_name'),col('b.categories'),
                                                           col('b.stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')])

In [0]:
def showInMap(df):
    
    mp = folium.Map(location=[36.1699, -115.1398], zoom_start=12)

    for i, r in df.toPandas().iterrows():
        folium.Marker(
                    location =[float(r.latitude), float(r.longitude)], 
                    popup = html.escape(r["business_name"]) + '<br>' + 'Stars: ' + str(r.stars) + '<br>' + 'Reviews: ' + str(r.review_count),    
                    icon = folium.Icon(color='orange')).add_to(mp)
    return mp

In [0]:
query = """
select business_id,user_id, count(*) as 4_5_stars_count
from reviews
where user_id in
    (select f.friend_id from friends f
    inner join users u on f.friend_id = u.user_id
    where f.user_id = "{}") 
and stars >= 4 
and business_id not in (select business_id from reviews where user_id = "{}")
group by business_id,user_id
order by count(*) desc limit 100
""".format(u_id, u_id)

friend_recoms_df = spark.sql(query)
print(friend_recoms_df.printSchema())
#friend_recoms_df = friend_recoms_df.withColumn(col('user_id'),u_id).limit(sim_bus_limit)
print(friend_recoms_df.count())

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- 4_5_stars_count: long (nullable = false)

None
26


In [0]:
def getFriendRecoms(u_id, sim_bus_limit=10):
    
    query = """
    select business_id, user_id,count(*) as 4_5_stars_count
    from reviews
    where user_id in
        (select f.friend_id from friends f
        inner join users u on f.friend_id = u.user_id
        where f.user_id = "{}") 
    and stars >= 4 
    and business_id not in (select business_id from reviews where user_id = "{}")
    group by business_id,user_id
    order by count(*) desc limit 100
    """.format(u_id, u_id)

    friend_recoms_df = spark.sql(query).limit(sim_bus_limit)
    
    return getBusinessDetails(friend_recoms_df)

In [0]:

u_id = 'ObQl16Vuc4sHPGKtzSGNUw'
result_df1 = getFriendRecoms(u_id)
print(result_df1.show())

+--------------------+--------------------+---------------+--------------------+--------------------+-----+------------+-------------+---------------+
|         business_id|             user_id|4_5_stars_count|       business_name|          categories|stars|review_count|     latitude|      longitude|
+--------------------+--------------------+---------------+--------------------+--------------------+-----+------------+-------------+---------------+
|d_L-rfS1vT3JMzgCU...|s-591-mtIyP7F1Lff...|              1|        Border Grill|Breakfast & Brunc...|  4.0|        1623|36.0900669068|-115.1769728974|
|XZbuPXdyA0ZtTu3Az...|s-591-mtIyP7F1Lff...|              1|          Mesa Grill|American (Traditi...|  4.0|        3268|36.1171245659| -115.174792707|
|K0j_Znzin0jShXVnp...|s-591-mtIyP7F1Lff...|              1|               Jaleo|American (New), R...|  3.5|        1221|    36.109889|    -115.174144|
|RESDUcs7fIiihp38-...|s-591-mtIyP7F1Lff...|              1|    Bacchanal Buffet|Restaurants, B

In [0]:
result_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- 4_5_stars_count: long (nullable = false)
 |-- business_name: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [0]:
result_df.coalesce(1).write.parquet("/content/drive/My Drive/Project /Data Mining/Dataset/Result/Final/friend_result.parquet")

In [0]:
showInMap(friend_recom_df)